In [1]:
# !nvidia-smi
# !pip -q install "transformers>=4.43.0" "trl>=0.9.6" "peft>=0.11.1" "accelerate>=0.34.2" bitsandbytes datasets sentencepiece

Tue Sep 23 23:01:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Target model
MODEL_NAME   = "mistralai/Mistral-7B-Instruct-v0.3"

# Output dirs
OUTPUT_DIR   = "mistral-dea-sft"     
MERGED_DIR   = OUTPUT_DIR + "-merged"  

# Tokenization / sequence
MAX_SEQ_LEN  = 2048

# QLoRA training hyperparams
BATCH_SIZE   = 1                     
GRAD_ACCUM   = 8
EPOCHS       = 1
LR           = 2e-4                 
LORA_R       = 16
LORA_ALPHA   = 32
LORA_DROPOUT = 0.05
SEED         = 42


In [3]:
raw_text = r"""
[1] front :: Abstract
Alcoholic liver disease (ALD) is a complicated disease which can lead to hepatocellular carcinoma; however, there is a lack of satisfactory therapeutics. Dehydroeburicoic acid (DEA) (1), a triterpenoid isolated from Antrodia cinnamomea, has been reported to act against ALD, but its mechanisms of action are still not clear. In this study, we report for the first time the use of DEA (1) as a dual inhibitor of the Keap1–Nrf2 protein–protein interaction (PPI) and GSK3β in an in vitro ALD cell model. DEA (1) engages Keap1 to disrupt the Keap1–Nrf2 PPI and inhibits GSK3β to restore Nrf2 activity in a Keap1-independent fashion. DEA (1) promotes Nrf2 nuclear translocation to activate downstream antioxidant genes. Importantly, DEA (1) restores the mitochondrial dysfunction induced by ethanol and generates antioxidant activity in the ALD cell model with minimal toxicity. We anticipate that DEA (1) could be a potential scaffold for the further development of clinical agents for treating ALD.
--------------------------------------------------------------------------------
[2] front :: Keywords
alcoholic liver disease (ALD), Keap1–Nrf2 protein–protein interaction (PPI), glycogen synthase kinase 3β (GSK3β), hepatoprotective
--------------------------------------------------------------------------------
[3] front :: Funding
Science and Technology Development Fund (Macau SAR, China)0007/2020/A10020/2022/A1State Key Laboratory of Quality Research in Chinese Medicine (University of Macau)SKL-QRCM(UM)-2020-2022University of Macau (University of Macau)MYRG2019-00002-ICMSMYRG2020-00017-ICMS2022 Internal Research Grant of SKL-QRCM (University of Macau)220771092177513182073715HKBU SKLEBA Research GrantSKLP_2223_P03This work is supported by the Science and Technology Development Fund (Macau SAR, China) (0007/2020/A1, 0020/2022/A1), the State Key Laboratory of Quality Research in Chinese Medicine (University of Macau) (SKL-QRCM(UM)-2020-2022), the University of Macau (University of Macau) (MYRG2019-00002-ICMS, MYRG2020-00017-ICMS), 2022 Internal Research Grant of SKL-QRCM (University of Macau) (QRCM-IRG2022-011), the National Natural Science Foundation of China, China (22077109, 21775131 and 82073715), the HKBU SKLEBA Research Grant (SKLP_2223_P03).
--------------------------------------------------------------------------------
[4] front :: Title
Dehydroeburicoic Acid, a Dual Inhibitor against Oxidative Stress in Alcoholic Liver Disease
--------------------------------------------------------------------------------
[5] body :: 1. Introduction
Excessive alcohol consumption leads to oxidative stress in the liver as a result of alcohol metabolism, resulting in inflammatory damage and the injury of liver cells []. Almost 20% of alcoholics are eventually diagnosed with alcoholic liver disease (ALD) []. The most common therapeutic options for ALD can be divided into opioid receptor antagonists, supplements to modulate liver metabolism, and therapeutics that regulate alcohol metabolism []. However, these treatments have been associated with side effects including dizziness, drug dependence, dermatitis, vomiting, and leukopenia []. Thus, there are currently no highly satisfactory therapeutic options for ALD, and hence there is an unmet clinical need to develop more effective and safer drugs for patients with ALD. Increasing evidence indicates that alcohol can damage mitochondria and destroy cellular homeostasis in the liver [,,]. One mechanism that cells use to defend against alcohol–induced injury is through the activation of nuclear respiratory factor-2 (Nrf-2). Nrf2 orchestrates a complex, self-protective antioxidant response involving numerous signaling axes []. Upregulating Nrf2 has been envisioned as an effective tool for combating alcohol-induced acute liver injury [,]. Normally, the level of Nrf2 in the cytosol is low due to ubiquitination and proteasomal degradation regulated by a Kelch-like ECH-associated protein (Keap1) [,]. However, the presence of oxidants can oxidize cysteine residues in Keap1 into disulfides or conjugate them to electrophiles, resulting in the nuclear translocation of Nrf2 and the activation of antioxidant genes [], including heme oxygenase-1 (HO–1), NAD(P)H dehydrogenase [quinone] 1 (NQO1), and the mitochondrial superoxide dismutase 2 (SOD2) [,]. In addition, Nrf2 activation increases the expression levels of nuclear respiratory factor-1 (Nrf-1) and peroxisome proliferator-activated receptor γ coactivator 1α (PGC1α), which in turn modulates the expression of mitochondrial respiratory subunits and translational components []. The genes activated by the antioxidant responsive element (ARE) protect cells against any damage caused by reactive oxygen species (ROS) []. In addition to Keap1, glycogen synthase kinase 3β (GSK3β) is another negative regulator of Nrf2 that acts to downregulate the antioxidant stress response [,]. GSK3β promotes the nuclear exclusion and degradation of Nrf2 in a Keap1-independent fashion in stressed or injured cells with active Nrf2, thereby reducing the degree of protection conferred by Nrf2 activity [,]. Therefore, increasing the Nrf2 antioxidant activity via targeting Keap1 or GSK3β could be a potential approach for treating ALD []. Most pharmacological approaches towards Nrf2 activation are focused on the inhibition of Keap1 [,]. Several Keap1 inhibitors have been reported in the last decade, and the vast majority of these covalently modify cysteine residues in Keap1 []. Covalently targeting cysteines in Keap1 may lack selectivity due to the presence of other reactive cysteine residues in the cell, resulting in adverse side effects [,]. However, because cysteine residues are abundant in cells, the safety and specificity of covalent drugs are a concern []. Recently, protein–protein interaction (PPI) inhibitors of Keap1 and Nrf2, developed based on the X-ray structure of Keap1, have emerged as a new class of Nrf2 activators [,,]. Our group has previously reported a cyclometalated iridium(III) metal complex as a Keap1–Nrf2 PPI inhibitor, which is a promising therapeutic agent for acetaminophen–induced acute liver injury [,]. However, no Keap1–Nrf2 inhibitor has yet entered the clinic for the treatment of human diseases [,]. Meanwhile, although a few GSK3β inhibitors have been reported for liver injury treatment, no GSK3β inhibitor to date has made it to the market []. Antrodia cinnamomea, a basidiomycete that is endemic to Taiwan, is an edible fungus that is a component of many traditional herbal medicines [,]. Either crude extracts or molecules isolated from A. cinnamomea have shown diverse biological properties, including antioxidant, anti-tumour, and anti-inflammatory activities [,]. Recently, research has shown that dehydroeburicoic acid (DEA) (1) from A. cinnamomea could inhibit alcoholic fatty liver disease (AFLD) by upregulating aldehyde dehydrogenase 2 family member (ALDH2) activity [,]. Additionally, DEA (1) also exhibited protective effects against non-alcoholic fatty liver disease (NAFLD) through activating ALDH2 and accelerating the elimination of ROS and harmful aldehydes []. Therefore, DEA (1) has the potential ability to treat ALD. However, its mechanisms of hepatoprotection in ALD are still not clear. Besides DEA (1), other triterpenoids have also been studied as GSK3β inhibitors and activators of Nrf2 because of their anti-inflammatory and antioxidant activities [,,,,]. Research has shown that single–molecule drugs with two different biological activities may avoid some side effects and improve efficacy []. In this study, we discovered a potent dual Keap1–Nrf2 PPI and GSK3β inhibitor, DEA (1), and investigated its underlying mechanisms of action. Through in vitro fluorescence polarization (FP) screening, we identified DEA (1) as the most potent candidate for disrupting the Keap1–Nrf2 PPI (EC50 = 14.1 µM) from thirteen analogues isolated from A. cinnamomea. Moreover, DEA (1) inhibited GSK3β kinase activity with an EC50 value of 8.0 µM. In cells, low-toxicity compound DEA (1) engaged Keap1 and GSK3β to promote Nrf2 accumulation in the nucleus, leading to the upregulation of ARE transcriptional activity and increasing the expression of downstream antioxidant factors with more potent than the reported Keap1–Nrf2 PPI inhibitor ML334 in ALD model cells.
--------------------------------------------------------------------------------
[6] body :: 2.1. DEA (1) Inhibits the Keap1–Nrf2 PPI and GSK3β Activity
The fluorescence polarization assay is a commonly used high-throughput assay to detect PPI inhibitors in solution. We used labeled Nrf2 peptides, bearing the high–affinity ETGE motif, which is recognized by Keap1, as fluorescent tracers to monitor for inhibition of the Keap1–Nrf2 interaction. ML334 is the first non–covalent small molecule inhibitor of Keap1–Nrf2 interaction reported and is distinct from other Nrf2 inducers []. In this paper, ML334 was used as a positive control. Compounds 1–13 from A. cinnamomea and ML334 were screened against the Keap1–Nrf2 interaction by FP (A). The preliminary screening results showed that DEA (1) was the most potent compound, which disrupted the Keap1–Nrf2 interaction by 62% inhibition at 50 µM (cf. ML334: 60% inhibition). The NMR spectrum of DEA (1) is presented in . The potency of DEA (1) was further investigated in a dose-response assay, revealing an EC50 value of 14.1 ± 0.1 µM against Keap1–Nrf2 PPI (B). We also explored the inhibitory effect of DEA (1) against GSK3β, which negatively regulates Nrf2 in a fashion independent of Keap1. 10 µM of DEA (1) reduced GSK3β activity by 58.9%, making it more potent than ML334 at the same concentration (11.3% inhibition) (C). Moreover, a dose-response assay revealed an EC50 value of 8.0 ± 0.7 µM against GSK3β activity, indicating that DEA (1) is an effective GSK3β inhibitor (D). In the nucleus, Nrf2 binds to the ARE to activate antioxidant gene transcription []. Thus, we tested the transcriptional activity of ARE after treatment of LO2 cells with DEA (1) and ML334 (10 µM) using the dual luciferase assay kit (E). The activity of ARE was significantly increased by 54% in the treatment of DEA (1) compared to 27% in the ML334 group. The results showed that DEA (1) notably activates the transcriptional activity of ARE compared to ML334 in cellulo.
--------------------------------------------------------------------------------
[7] body :: 2.2. DEA (1) Exhibits Low Cytotoxicity In Cellulo
The cytotoxicity of compound DEA (1) was detected in both human normal cell lines LO2 and HEK 293T, and the hepatocellular carcinoma cell line HepG2. After treatment with different concentrations of DEA (1), no significant cytotoxicity was observed (F–H). These data indicate that DEA (1) could be potentially safe for treating ALD in vivo.
--------------------------------------------------------------------------------
[8] body :: 2.3. DEA (1) as a Dual Inhibitor of Keap1 and GSK3β
The ability of DEA (1) to disrupt the Keap1–Nrf2 binding in cellulo was monitored through a Co–IP experiment using human normal liver cell line LO2 cells (A). After treatment with DEA (1) (10 µM) or ML334 (10 µM) for 8 h, there was a 45% reduction of Keap1 co-precipitated with Nrf2 for DEA (1) as compared to 12% inhibition for ML334, indicating that DEA (1) could more effectively inhibit the Keap1–Nrf2 interaction compared to ML334 in living cells (B). We investigated the ability of DEA (1) to target Keap1, Nrf2, and GSK3β in the cellular environment using CETSA. LO2 cell lysates were incubated with 10 µM of DEA (1) at room temperature for 30 min. Then, aliquots were heated individually at different temperatures and the protein in the soluble fraction was quantified by WB. Compound DEA (1) significantly stabilized Keap1 (ΔTm: ca. 4.5 °C) and GSK3β (ΔTm: ca. 3.9 °C), while having no observable effect on the thermal stability of Nrf2 and β–actin (C,D). This result suggests that DEA (1) can bind with Keap1 and GSK3β, even within the complicated environment of cell lysates.
--------------------------------------------------------------------------------
[9] body :: 2.4. DEA (1) Targets Keap1 and GSK3β to Induce Nrf2 Accumulation in the Nucleus and the Expression of Downstream Antioxidant Proteins
To verify whether Keap1 or GSK3β are direct targets of DEA (1), knockdown assays were performed. NQO1 and HO–1 are target antioxidant genes of Nrf2 that increase cytoprotection against oxidative stress [] As expected, Keap1 knockdown in LO2 cells induced a noticeable rise of the expression levels of both Nrf2 and its gene targets, NQO1 and HO–1 (A,B). However, treating Keap1 knockdown cells with DEA (1) led to no further rises in the NQO1 and HO–1 level, in contrast to the control cells, where DEA (1) treatment produced noticeable increases of NQO1 and HO–1. These results suggest that DEA (1) activates NQO1 and HO–1 through targeting Keap1. Similarly, GSK3β knockdown induced Nrf2 and downstream antioxidant protein expression but without influencing Keap1 protein level (C,D). In addition, DEA (1) had a diminished effect on NQO1 and HO–1 level in GSK3β knockdown cells as compared to the control cells. Finally, we knocked down GSK3β and Keap1 at the same time to verify the dual binding mechanism of DEA (1) on GSK3β and Keap1 (E,F). The results showed that treating the GSK3β and Keap1 knockdown cells with DEA (1) did not cause a further significant induction of NQO1 and HO–1 expression. Evidence for the hypothesis that DEA (1) acts through targeting both Keap1 and GSK3β comes from considering the Nrf2 level. In either the Keap1 knockdown or GSK3β knockdown cells, DEA (1) treatment resulted in slight increases of Nrf2 expression relative to DMSO. However, in the double Keap1 and GSK3β knockdown cells, treatment with DEA (1) did not lead to increases of Nrf2 expression. These results suggest that DEA (1) treatment mimicked the effects of double knockdown in increasing Nrf2 expression. Therefore, DEA (1) exerts its antioxidant effects by targeting both Keap1 and GSK3β. Inhibition of the Keap1–Nrf2 interaction is known to increase Nrf2 accumulation in the nucleus []. Meanwhile, GSK3β is also a negative regulator of Nrf2 accumulation by acting independently of Keap1 []. Therefore, we evaluated the Nrf2 level in the cytoplasm and nucleus of LO2 cells via WB after the treatment of cells with DEA (1) (10 µM) for 8 h. As shown in A, DEA (1) significantly increased Nrf2 translocation into the nucleus in a dose-dependent manner. GSK3β knockdown also phenocopied compound DEA (1) treatment at inducing Nrf2 nuclear accumulation without influencing Keap1 level (B). In summary, DEA (1) disrupts the Keap1–Nrf2 PPI and inhibits GSK3β to increase the translocation of liberated Nrf2 into the nucleus, thereby presumably increasing ARE transcriptional activity.
--------------------------------------------------------------------------------
[10] body :: 2.5. DEA (1) Activates Nrf2 Downstream Antioxidant Genes in ALD Model Cells
Nuclear Nrf2 heterodimerizes with musculoaponeurotic fibrosarcoma (Maf) to recognize ARE and promote the transcription of antioxidant genes, including Nrf1, HO–1, and NQO1 []. Therefore, we evaluated the ability of DEA (1) to increase Nrf1, NQO1, and HO–1 levels in ALD model cells, which were prepared by treating LO2 cells with 0.3% EtOH for 8 h. 0.3% EtOH was reported to induce oxidative stress in living cells [,,]. In addition, a preliminary time-dependent assay was performed, which identified an optimal incubation time length of 8 h (). As shown in A, DEA (1) (30 μM) induced Nrf1 (B), NQO1 (C) and HO–1 (D) protein levels by 1.2, 3.8 and 1.8-fold, respectively, in ALD model cells, making it more potent than ML334 (30 μM), which increased Nrf1, NQO1, and HO–1 protein levels by 1.0, 1.1, and 1.5-fold, respectively. Nrf2 directly controls the expression of the transcriptional factor PGC–1α, a regulator of mitochondrial biogenesis, while it also directly regulates the expression of mitochondrial antioxidant enzymes, such as superoxide dismutase 2 (SOD2) [,]. Hence, the effect of DEA (1) on mRNA levels of SOD2, HO–1, NQO1, Nrf1, and PGC–1α in 0.3% EtOH–treated LO2 cells was also evaluated. 30 μM of DEA (1) rescued SOD2 mRNA level in ALD model cells by 2-fold, compared to 1.5-fold with ML334 (E). DEA (1) (30 μM) also increased NQO1 (F) and HO–1 (G) mRNA levels by 2.0 and 2.5-fold, respectively, making it more potent than ML334 which increased NQO1 and HO–1 mRNA levels by 1.8 and 1.4-fold, respectively. In addition, the mRNA levels of Nrf1 and PGC–1α were also increased 2-fold by compound DEA (1) (30 μM) (H,I). The results also show a slight increase of NQO1, HO–1, and PGC–1α levels by 0.3% EtOH treatment alone, which has been previously observed in other studies []. However, the subsequent increases of antioxidant genes were much greater upon further treatment of DEA (1). Taken together, these results indicate that DEA (1) can activate Nrf2 downstream antioxidant genes in EtOH–treated human liver cells, a cellular model of ALD.
--------------------------------------------------------------------------------
[11] body :: 2.6. DEA (1) Repairs the EtOH Induced Mitochondrial Dysfunction and Improves the Antioxidant Activity in LO2 Cells
Inflammation and oxidative stress increase ROS production, and the activated Nrf2 was able to maintain low intracellular ROS level to protect the cell from oxidant injury []. Moreover, growing evidence shows that mitochondrial and oxidant injury induced by ethanol consumption play key roles in alcohol–induced liver injury []. Alcohol–induced ROS causes mitochondrial membrane depolarization and mitochondrial permeability transition (MPT), leading to hepatic apoptosis and necrosis []. Therefore, we detected the effect of DEA (1) and ML334 on ROS level in 0.3% EtOH–treated LO2 cells (A). The results indicated that both 1 and ML334 could significantly decrease ROS level induced by EtOH. The MPTP assay showed that DEA (1) (30 μM) can restore MPTP function (40%) with a higher activity as compared to ML334 (34%) (B,C). ATP is an indicator of mitochondrial function and ATP level are significantly reduced in ethanol–fed Nrf2−/− mice []. DEA (1) (30 μM) rescued 50% of ATP in LO2 cells after EtOH treatment, making it more potent than ML334, which rescued 20% of the ATP level under the same conditions (D). Finally, DEA (1) (30 μM) induced a higher increase of antioxidant activity (4-fold) compared to ML334 (2-fold) under the same conditions (E).
--------------------------------------------------------------------------------
[12] body :: 3. Discussion
ALD is a serious chronic liver disease caused by oxidative stress and alcohol metabolism, which leads to massive global deaths []. Unfortunately, until now, there are no effective therapeutic drugs approved by the FDA for the treatment of ALD []. The pathogenesis of ALD is poorly characterized and research on the mechanism is still not clear. A number of reports have shown that the Keap1–Nrf2 pathway plays an important role in activating ARE antioxidant signaling []. Nrf2 is regulated via Keap1-independent pathways, including mitogen-activated protein kinase–Erk and PI3K–Akt pathways []. Among these regulatory pathways, GSK3β has emerged as a convergent point [,]. Because Nrf2 degradation and nuclear exclusion are regulated by GSK3β, it is critical in switching off the self–protective antioxidant stress response after injury [,]. Hence, targeting Keap1 and GSK3β to activate Nrf2 are potential therapeutic strategies for ALD. Natural products provide diverse scaffolds with high bioactivity and low toxicity, and could be developed as promising candidates in drug development []. A. cinnamomea is a traditional Chinese herb that is a component of various medicines []. Triterpenoids are the major constituent in A. cinnamomea, and several studies have demonstrated their antioxidative and hepatoprotective effects [,]. However, the hepatoprotective mechanism of these compounds has not been extensively explored. DEA (1) has previously shown promising hepatoprotective activity in some studies [], including in mouse models [,,,]. In recent years, many Keap1–Nrf2 PPI inhibitors and GSK3β inhibitors have been reported for various indications, such as Alzheimer’s disease, cancers, Parkinson’s disease, kidney diseases, Acetaminophen (APAP)–induced liver injury, and ferroptosis [,,,,,]. However, few have been studied in the context of ALD. In our present study, we discovered DEA (1) as a potent Keap1–Nrf2 PPI inhibitor from A. cinnamomea extracts. We explored the detailed mechanism of hepatoprotective action of DEA (1) in ALD, which acted via targeting both the Keap1–Nrf2 PPI and GSK3β (). To our best knowledge, the dual inhibition mechanism of DEA (1) in ALD has not been reported before. In in vitro assays, DEA (1) inhibited the Keap1–Nrf2 PPI (EC50 = 14.1 µM) as well as GSK3β kinase activity (EC50 = 8.0 ± 0.7 µM) with greater potency than the known Keap1–Nrf2 inhibitor, ML334. Cellular experiments confirmed the ability of DEA (1) to engage Keap1 and GSK3β in cellulo to activate Nrf2 and stimulate the expression of ARE–controlled genes. The specificity of DEA (1) was further confirmed using Keap1 and GSK3β knockdown experiments. DEA (1) treatment phenocopied dual knockdown of Keap1 and GSK3β, suggesting that DEA (1) acted simultaneously through both pathways to exert its biological effects. Moreover, DEA (1) increased Nrf2 nuclear accumulation, mimicking GSK3β knockdown, without influencing Keap1 level. Finally, we showed that DEA (1) could restore MPTP function and reduce ROS level induced by EtOH in ALD model cells. DEA (1) displayed low cytotoxicity in both normal and cancerous liver cells, indicating its potential safety for treating ALD in vivo. In summary, DEA (1) is a promising scaffold for the further development of dual inhibitors of the Keap1–Nrf2 PPI and GSK3β as therapeutic agents against ALD. The structural modification of compound DEA (1) is in progress and animal studies will be performed in due course.
--------------------------------------------------------------------------------
[13] body :: 4.1. Cell Lines and Culture
The human normal liver cell line (LO2), human embryonic kidney 293 T cell line, and HepG2 cell line were purchased from American Type Culture Collection (Manassas, VA, USA). Cells were cultured in Dulbecco’s modified Eagle’s medium (DMEM), with 10% FBS, 100 units/mL penicillin, 100 μg/mL streptomycin. Fetal bovine serum (FBS), DMEM, penicillin, and streptomycin were purchased from Gibco BRL (Gaithersburg, MD, USA).
--------------------------------------------------------------------------------
[14] body :: 4.2. Chemical
Compounds 1–13 (purity > 98% by HPLC analysis) were isolated from A. cinnamomea in a previous study []. New 1H and 13C nuclear magnetic resonance (NMR) were recorded for DEA (1) to verify its identity for this study (). Spectral data were consistent with our previous report [].
--------------------------------------------------------------------------------
[15] body :: 4.3. Fluorescence Polarization Assay
The ability of compounds 1–13 and ML334 to inhibit Keap1–Nrf2 peptide binding was evaluated using a FP assay according to the manufacturer’s instruction (BPS Bioscience, San Diego, CA, USA). ML334 was a non–covalent small molecule inhibitor of the Keap1–Nrf2 and purchased from MedChemExpress (Princeton, NJ, USA). The tested compounds were dissolved in the provided buffer at the indicated concentrations. Then, the required volumes of Nrf2 peptide, Bovine Serum Albumin (BSA), Keap1 protein, and assay buffer were added to each well, followed by incubation for 30 min and the fluorescent polarization value was recorded.
--------------------------------------------------------------------------------
[16] body :: 4.4. Western Blot (WB) and Co–Immunoprecipitation (Co–IP)
The protocol was reported in a previous study []. In brief, LO2 cells were seeded at a density of 2 × 106 cells in a six–well plate. LO2 lysate is harvested after incubation with 10 μM of compound DEA (1) or ML334 for 8 h. A 10% SDS–PAGE gel was used to separate the LO2 cell lysates, which were transferred to a PVDF membrane (Bio–Rad, Hercules, CA, USA). The membranes were incubated with antibodies anti–Keap1 (1:1000, Cell Signaling Technology, Danvers, MA, USA; Cat# 8047S), Nrf2 (1:1000, Proteintech, Wuhan, China; Cat# 16396-1-AP), GSK3β (1:1000, Cell Signaling Technology, Danvers, MA, USA; Cat# 9832S), HO–1 (1:1000, Cell Signaling Technology, Danvers, MA, USA; Cat# 5853S), NQO1 (1:1000, Cell Signaling Technology, Danvers, MA, USA; Cat# 3187S), β–actin (1:1000, Absin, Shanghai, China; Cat#: abs137975), or anti-lamin B (1:1000, Cell Signaling Technology, Danvers, MA, USA; Cat # 12586S) overnight. The membranes were washed with wash buffer three times. A secondary antibody (1:1000) was added to the membranes. After incubated for 2 h, the proteins bands were detected using enhanced chemiluminescent Plus reagents (GE Healthcare, Boston, MA, USA) and analyzed by Image Lab. The antibodies were purchased from Abcam (Waltham, MA, USA). Co–IP between Keap1 and Nrf2 was performed following the protocol from Life Technologies. Briefly, lysates from 2 × 106 LO2 cells were incubated with anti–Nrf2 or Rabbit mAb IgG (Abcam, Waltham, MA, USA; Cat#: ab205718) overnight. Then, samples were incubated with 10 μL Dynabeads (Life technologies, Foster, CA, USA) for 6 h. The Dynabeads were washed and analyzed by WB with the Keap1 antibody.
--------------------------------------------------------------------------------
[17] body :: 4.5. Real-Time Quantitative Polymerase Chain Reaction (RT–qPCR)
LO2 cells were incubated with 10 µM or 30 µM compound DEA (1) for 8 h and the total RNA was extracted using the NucleoSpin® RNA Plus kit (Takara, Tokyo, Japan). We checked the RNA quality during the RT–qPCR operation and ensured that all the A260/280 value of samples were around 2, which is consistent with the literature []. We also performed an RT negative control and found no PCR amplification. Then, cDNA was synthesized using the PrimeScript™ RT Reagent Kit (Takara, Tokyo, Japan). The RT–qPCR conditions were as previously reported, with minor modifications []. DNA denaturation temperature: 95 °C (30 s); annealing temperature: 58 °C (30 s); extension temperature: 72 °C (45 s), number of cycles: 40. RT–qPCR was performed and analyzed as described previously []. The related primers are listed in .
--------------------------------------------------------------------------------
[18] body :: 4.6. MTT Assay
The LO2, HUVEC and HEK 293 T cell lines were seeded at 1 × 104 cells per well in 96–well plates and incubated with DMEM overnight at 37 °C in a humidified CO2 incubator to ensure attachment. The medium was then replaced with a medium that contained the different concentrations of compound for 48 h at 37 °C. The final concentration of DMSO was 0.1% or lower in all cases (including controls). The medium with 100 μL of MTT (3-(4,5-dimethylthiazol-2-yl)-2,5-tetrazolium bromide) reagent (1 mg/mL) was then replaced. After 4 h incubation, it was replaced with 100 μL of DMSO and measured using a microplate reader at 570 nm.
--------------------------------------------------------------------------------
[19] body :: 4.7. Detection of ROS and Antioxidant Activity
LO2 cells at a density of 2 × 106 cells/well in a six-well plate were treated with different concentrations of DEA (1) or 10 μM ML334 for 8 h. ROS level and antioxidant activity were measured using the Reactive Oxygen Species Assay kit and the Total Antioxidant Capacity Assay Kit with FRAP, respectively, according to the manufacturer’s instructions (Beyotime, Shanghai, China).
--------------------------------------------------------------------------------
[20] body :: 4.8. Nuclear and Cytoplasmic Extraction
LO2 cells were incubated with 10 µM or 30 µM of DEA (1) for 8 h. The nuclear and cytoplasmic extraction followed the protocol from the manufacturer (Epigentek, New York, NY, USA, EpiQuik™ Nuclear Extraction Kit I, Catalog # OP-0002), Then, protein expression was analyzed by WB with the indicated antibodies.
--------------------------------------------------------------------------------
[21] body :: 4.9. GSK3β Kinase Assay
The ability of DEA (1) and ML334 to inhibit GSK3β activity was evaluated using luminescent kinase assay according to the manufacturer’s instruction (Promega Corporation, Madison, WI, USA). Briefly, the 384 well white plate was mixed with 1 μL of inhibitor, 2 μL of enzyme, and 2 μL of substrate/ATP mix and incubated at room temperature for 60 min. After 60 min, 5 μL of ADP–Glo™ reagent was added and incubated at room temperature for 40 min. Then 10 μL of kinase detection reagent was added, incubated at room temperature for 30 min and the luminescence value was recorded (integration time 0.5–1 s).
--------------------------------------------------------------------------------
[22] body :: 4.10. Dual Luciferase Reporter Gene Assay
LO2 cells were seeded in a six–well plate with 80% confluence in a DMEM medium for 24 h. The ARE–luciferase vector or a control plasmid with TurboFect transfection reagent (Thermo Fisher Scientific, Waltham, MA, USA) were transfected into LO2 cells according to the manufacturer’s instruction, and cells were incubated for 48 h. Then, LO2 cells were incubated with 10 µM compound DEA (1), 10 µM ML334 or DMSO for 4 h. The ARE activity was detected through the manufacturer’s instruction (Beyotime Dual Luciferase Reporter Gene Assay Kit, Shanghai, China).
--------------------------------------------------------------------------------
[23] body :: 4.11. Cellular Thermal Shift Assay (CETSA)
LO2 cell lysates were incubated with 10 μM compound DEA (1) and DMSO at room temperature for 30 min. The lysates are divided and heated individually at the indicated temperature. The supernatants of the heated lysates were collected after centrifugation, and then analyzed by WB with the Keap1, Nrf2, and β–actin antibodies.
--------------------------------------------------------------------------------
[24] body :: 4.12. Mitochondrial Permeability Transition Pore (MPTP) Assay
MPTP kit was used for evaluation of mitochondrial permeability transition according to the manufacturer’s instructions (Beyotime, Shanghai, China). A density of 1 × 104 cells in a six–well plate, EtOH–induced LO2 cells were treated with 10 µM of ML334, 10 or 30 µM compound DEA (1), DMSO for 8 h, and then pretreated with calcein or calcein and CoCl2. Cells were stained by calcein acetoxymethyl ester (Calcein AM) and MPT function was detected by fluorescence spectroscopy.
--------------------------------------------------------------------------------
[25] body :: 4.13. SiRNA Gene Knockdown
The sequences of Keap1 siRNA are 5′-GGCCUUUGGCAUCAUGAACTT-3′ (sense) and 5′-GUUCAUGAUGCCAAAGGCCTG-3′ (antisense) []. GSK3β siRNA is 5′-GCAUUUAUCGUUAACCUAA-3′ (sense) and 5′-UUAGGUUAACGA UAAAUGC-3′ (antisense) []. Keap1 and GSK3β siRNA was transfected with Lipo3000 reagent (Thermo Fisher Scientific, Waltham, MA, USA) into 1 × 106 LO2 cells following the manufacturer’s instruction. After incubation for 48 h, Keap1 and GSK3β protein expression and functional experiments were performed.
--------------------------------------------------------------------------------
[26] body :: 4.14. Statistical Analysis
Statistical significance was determined using the Student’s t-test for experiments comparing two groups. The data is homogeneous and follows a normal distribution before proceeding to the statistical analysis. Comparisons among groups were analysed using analysis of variance (ANOVA). All statistical tests were done using the GraphPad Prism version 8.0 software (GraphPad Software Inc. San Diego, CA, USA). All values were expressed as the mean ± standard deviation (SD). p < 0.05 means statistical significance.
--------------------------------------------------------------------------------
[27] body :: 5. Conclusions
Recent studies on ALD are scarce compared to research on other liver diseases, and there are still no highly satisfactory therapeutic options for ALD. To our knowledge, no dual inhibitor of the Keap1–Nrf2 PPI and GSK3β to treat alcohol-induced liver injury has been reported. This paper has revealed that DEA (1), one of the active triterpenes isolated from A. cinnamomea, could be a potential lead compound for treating ALD. Compound DEA (1) inhibited the Keap1–Nrf2 interaction and GSK3β activity both in vitro and in cellulo, and displayed low cytotoxicity at the concentrations required for inducing antioxidant activity. Moreover, DEA (1) could engage Keap1 and GSK3β in liver cells and increase Nrf2 nucleus translocation, thereby activating ARE transcriptional activity and upregulating antioxidant genes and mitochondrial biogenesis regulators. Significantly, compound DEA (1) displayed more potent hepatoprotective activity when compared with the classic Keap1–Nrf2 PPI inhibitor ML334. Therefore, the structure of DEA (1) may be optimized to expand the diversity, improve the activity, and be used in animal research. To our knowledge, this paper firstly reports DEA (1) acts as the dual inhibitor of Keap1–Nrf2 PPI and GSK3β with the potential for protecting against ALD. We anticipate that natural product DEA (1) can be considered as a potential scaffold for the development of clinical agents for treating ALD.
--------------------------------------------------------------------------------
[28] back :: References
1.AvilaM.A.DufourJ.F.GerbesA.L.ZoulimF.BatallerR.BurraP.Cortez-PintoH.GaoB.GilmoreI.MathurinP.Recent advances in alcohol-related liver disease (ALD): Summary of a Gut round table meetingGut20196976478010.1136/gutjnl-2019-31972031879281PMC7236084 2.BakhautdinB.DasD.MandalP.RoychowdhuryS.DannerJ.BushK.PollardK.KasparJ.W.LiW.SalomonR.G.Protective role of HO-1 and carbon monoxide in ethanol-induced hepatocyte cell death and liver injury in miceJ. Hepatol.2014611029103710.1016/j.jhep.2014.06.00724946281PMC4203703 3.Higuera-de TijeraF.Servín-CaamañoA.Serralde-ZúñigaA.E.Cruz-HerreraJ.Pérez-TorresE.Abdo-FrancisJ.M.Salas-GordilloF.Pérez-HernándezJ.L.Metadoxine improves the three-and six-month survival rates in patients with severe alcoholic hepatitisWorld J. Gastroenterol.2015214975498510.3748/wjg.v21.i16.497525945012PMC4408471 4.TriantafyllouK.VlachogiannakosJ.LadasS.D.Gastrointestinal and liver side effects of drugs in elderly patientsBest Pract. Res. Clin. Gastroenterol.20102420321510.1016/j.bpg.2010.02.00420227033 5.LiX.X.JiangZ.H.ZhouB.ChenC.Hepatoprotective effect of gastrodin against alcohol-induced liver injury in miceJ. Physiol. Biochem.201975293710.1007/s13105-018-0647-830242628 6.NassirF.IbdahJ.A.Role of mitochondria in alcoholic liver diseaseWorld J. Gastroenterol.2014202136234210.3748/wjg.v20.i9.213624605012PMC3942818 7.AbdallahM.A.SingalA.K.Mitochondrial dysfunction and alcohol-associated liver disease: A novel pathway and therapeutic targetSignal Transduct. Target. Ther.202051210.1038/s41392-020-0128-832296016PMC7054548 8.FangY.ZhaoY.HeS.GuoT.S.SongQ.GuoN.YuanZ.Y.Overexpression of FGF19 alleviates hypoxia/reoxygenation-induced injury of cardiomyocytes by regulating GSK-3β/Nrf2/ARE signalingBiochem. Biophys. Res. Commun.20185032355236210.1016/j.bbrc.2018.06.16129964017 9.HanD.YbanezM.D.JohnsonH.S.McDonaldJ.N.MesropyanL.SanchetiH.MartinG.MartinA.LimA.M.DaraL.Dynamic adaptation of liver mitochondria to chronic alcohol feeding in mice biogenesis, remodeling, and functional alterationsJ. Biol. Chem.2021287421654217910.1074/jbc.M112.377374PMC351676223086958 10.SongX.CuiW.MengF.XiaQ.LiX.HouM.JiaL.ZhangJ.Glucopyranose from Pleurotus geesteranus prevent alcoholic liver diseases by regulating Nrf2/HO-1-TLR4/NF-κB signalling pathways and gut microbiotaFood Funct.2022132441245510.1039/D1FO03486C35048917 11.JiangZ.Y.LuM.C.YouQ.D.Discovery and development of Kelch-like ECH-associated protein 1. nuclear factor erythroid 2-related factor 2 (KEAP1: NRF2) protein–protein interaction inhibitors: Achievements, challenges, and future directionsJ. Med. Chem.201659108371085810.1021/acs.jmedchem.6b0058627690435 12.ZhouJ.ZhengQ.ChenZ.The Nrf2 Pathway in Liver DiseasesFront. Cell Dev. Biol.20221082620410.3389/fcell.2022.82620435223849PMC8866876 13.ItohK.WakabayashiN.KatohY.IshiiT.IgarashiK.EngelJ.D.YamamotoM.Keap1 represses nuclear activation of antioxidant responsive elements by Nrf2 through binding to the amino-terminal Neh2 domainGenes Dev.199913768610.1101/gad.13.1.769887101PMC316370 14.MichaličkováD.HrnčířT.CanováN.K.SlanařO.Targeting Keap1/Nrf2/ARE signaling pathway in multiple sclerosisEur. J. Pharmacol.202087317297317298810.1016/j.ejphar.2020.17297332017935 15.MansouriA.ReinerŽ.RuscicaM.Tedeschi-ReinerE.RadbakhshS.Bagheri EktaM.SahebkarA.Antioxidant Effects of Statins by Modulating Nrf2 and Nrf2/HO-1 Signaling in Different DiseasesJ. Clin. Med.202211131310.3390/jcm1105131335268403PMC8911353 16.MitsuishiY.TaguchiK.KawataniY.ShibataT.NukiwaT.AburataniH.YamamotoM.MotohashiH.Nrf2 redirects glucose and glutamine into anabolic pathways in metabolic reprogrammingCancer Cell201222667910.1016/j.ccr.2012.05.01622789539 17.ZhangW.XiongH.PangJ.SuZ.W.LaiL.LinH.Q.JianB.Q.HeW.H.YangH.D.ZhengY.Q.Nrf2 activation protects auditory hair cells from cisplatin-induced ototoxicity independent on mitochondrial ROS productionToxicol. Lett.202033111010.1016/j.toxlet.2020.04.00532428544 18.XuD.XuM.JeongS.S.QianY.H.WuH.L.XiaQ.KongX.N.The role of Nrf2 in liver disease: Novel molecular mechanisms and therapeutic approachesFront. Pharmacol.201981428143510.3389/fphar.2018.01428PMC633145530670963 19.HuangS.WangY.XieS.LaiY.MoC.ZengT.KuangS.DengG.ZhouC.ChenY.Hepatic TGFβr1 Deficiency Attenuates Lipopolysaccharide/D-Galactosamine-Induced Acute Liver Failure Through Inhibiting GSK3β-Nrf2-Mediated Hepatocyte Apoptosis and FerroptosisCell Mol. Gastroenterol. Hepatol.2022131649167210.1016/j.jcmgh.2022.02.00935202887PMC9046809 20.ShuG.QiuY.HaoJ.FuQ.DengX.γ-Oryzanol alleviates acetaminophen-induced liver injury: Roles of modulating AMPK/GSK3β/Nrf2 and NF-κB signaling pathwaysFood Funct.2019106858687210.1039/C9FO01808E31584590 21.RojoA.I.Medina-CamposO.N.RadaPP.Zúñiga-ToalácA.López-GazcónA.EspadaS.Pedraza-ChaverriJ.CuadradoaA.Signaling pathways activated by the phytochemical nordihydroguaiaretic acid contribute to a Keap1-independent regulation of Nrf2 stability: Role of glycogen synthase kinase-3Free Radic. Biol. Med.20125247348710.1016/j.freeradbiomed.2011.11.00322142471 22.ChenS.ZouL.LiL.WuT.The protective effect of glycyrrhetinic acid on carbon tetrachloride-induced chronic liver fibrosis in mice via upregulation of Nrf2PLoS ONE20138e5366210.1371/journal.pone.005366223341968PMC3544925 23.YamamotoM.KenslerT.W.MotohashiH.The KEAP1-NRF2 system: A thiol-based sensor-effector apparatus for maintaining redox homeostasisPhysiol. Rev.2018981169120310.1152/physrev.00023.201729717933PMC9762786 24.MundalS.B.RaknerJ.J.SilvaG.B.GiermanL.M.AustdalM.BasnetP.ElschotM.BakkeS.S.OstropJ.ThomsenL.C.V.Divergent Regulation of Decidual Oxidative-Stress Response by NRF2 and KEAP1 in Preeclampsia with and without Fetal Growth RestrictionInt. J. Mol. Sci.202210196610.3390/ijms2304196635216082PMC8875334 25.LibyK.T.YoreM.M.SpornM.B.Triterpenoids and rexinoids as multifunctional agents for the prevention and treatment of cancerNat. Rev. Cancer2007735736910.1038/nrc212917446857 26.LuM.ZhouH.S.YouQ.D.JiangZ.Design, synthesis, and initial evaluation of affinity-based small-molecule probes for fluorescent visualization and specific detection of Keap1J. Med. Chem.2016597305731010.1021/acs.jmedchem.6b0077527409246 27.KimS.Indu ViswanathA.N.ParkJ.H.LeeH.E.ParkA.Y.ChoiJ.W.KimH.J.LondheA.M.JangB.K.LeeJ.Nrf2 activator via interference of Nrf2-Keap1 interaction has antioxidant and anti-inflammatory properties in Parkinson’s disease animal modelNeuropharmacology202016710798910.1016/j.neuropharm.2020.10798932032607 28.CuadradoA.RojoA.I.WellsG.HayesJ.D.CousinS.P.RumseyW.L.AttucksO.C.FranklinF.LevonenA.L.KenslerT.W.Therapeutic targeting of the NRF2 and KEAP1 partnership in chronic diseasesNat. Rev. Drug Discov.20191829531710.1038/s41573-018-0008-x30610225 29.JiangZ.Y.XuL.L.LuM.C.ChenZ.Y.YuanZ.W.XuX.L.GuoX.K.ZhangX.J.SunH.P.YouQ.D.Structure–activity and structure–property relationship and exploratory in vivo evaluation of the nanomolar Keap1–Nrf2 protein–protein interaction inhibitorJ. Med. Chem.2015586410642110.1021/acs.jmedchem.5b0018526258437 30.LiG.D.LiuH.FengR.B.KangT.S.WangW.H.KoC.N.WongC.Y.YeM.MaD.L.WanJ.B.A bioactive ligand-conjugated iridium (III) metal-based complex as a Keap1–Nrf2 protein-protein interaction inhibitor against acetaminophen-induced acute liver injuryRedox Biol.20214810212910.1016/j.redox.2021.10212934526248PMC8710994 31.SunC.HanB.ZhaiY.ZhaoH.LiX.QianJ.HaoX.LiuQ.ShenJ.KaiG.Dihydrotanshinone I inhibits ovarian tumor growth by activating oxidative stress through Keap1-mediated Nrf2 ubiquitination degradationFree Radic. Biol. Med.20222022023510.1016/j.freeradbiomed.2022.01.01535074488 32.LeungC.H.ZhangJ.T.YangG.J.LiuH.HanQ.B.MaD.L.Emerging Screening Approaches in the Development of Nrf2–Keap1 Protein–Protein Interaction InhibitorsInt. J. Mol. Sci.2019204445446410.3390/ijms2018444531509940PMC6770765 33.FengR.B.WangY.HeC.W.YangY.WanJ.B.Gallic acid, a natural polyphenol, protects against tert-butyl hydroperoxide-induced hepatotoxicity by activating ERK-Nrf2-Keap1-mediated antioxidative responseFood Chem. Toxicol.201811947948810.1016/j.fct.2017.10.03329066411 34.NooriM.S.BhattP.M.CourregesM.C.GhazanfariD.CucklerC.OracC.M.McMillsM.C.SchwartzF.L.DeosarkarS.P.BergmeierS.C.Identification of a novel selective and potent inhibitor of glycogen synthase kinase-3Am. J. Physiol.-Cell Physiol.2019317C1289C130310.1152/ajpcell.00061.201931553649PMC6962522 35.ChangT.T.ChouW.N.Antrodia cinnamomea reconsidered and A. salmonea sp. nov. on Cunninghamia konishii in TaiwanBot. Bull. Acad. Sin.200445347352 36.LiH.X.WangJ.J.LuC.L.GaoY.J.GaoL.YangZ.Q.Review of Bioactivity, Isolation, and Identification of Active Compounds from Antrodia cinnamomeaBioengineering2022949410.3390/bioengineering910049436290462PMC9598228 37.LiB.KuangY.HeJ.B.TangR.XuL.L.LeungC.H.MaD.L.QiaoX.YeM.Antcamphorols A-K, Cytotoxic and ROS Scavenging Triterpenoids from Antrodia camphorataJ. Nat. Prod.202083455410.1021/acs.jnatprod.9b0058031891260 38.YangX.WangX.LinJ.LimS.CaoY.ChenS.XuP.XuC.ZhengH.FuK.C.Structure and Anti-Inflammatory Activity Relationship of Ergostanes and Lanostanes in Antrodia cinnamomeaFoods202211183110.3390/foods1113183135804645PMC9266224 39.XuL.PengA.K.CaoY.N.QiaoX.YueS.S.YeM.QiR.Protective Effects of Antrodia cinnamomea and Its Constituent Compound Dehydroeburicoic Acid 32 against Alcoholic Fatty Liver DiseaseCurr. Mol. Pharmacol.20211487188210.2174/187446721466621012015214033494689 40.CaoY.N.YueS.S.WangA.Y.XuL.HuY.T.QiaoX.WuT.Y.YeM.WuY.C.QiR.Antrodia cinnamomea and its compound dehydroeburicoic acid attenuate nonalcoholic fatty liver disease by upregulating ALDH2 activityJ. Ethnopharmacol.20222811514610.1016/j.jep.2022.11514635304272 41.ToC.RoyA.ChanE.PradoM.A.M.GuglielmoG.M.D.Synthetic triterpenoids inhibit GSK3β activity and localization and affect focal adhesions and cell migrationBiochim. Biophys. Acta-Mol. Cell Res.201718641274128410.1016/j.bbamcr.2017.03.01228366661 42.JacobsK.M.BhaveS.R.FerraroD.J.JaboinJ.J.HallahanD.E.ThotalaD.GSK-3β: A Bifunctional Role in Cell Death PathwaysInt J Cell Biol.2012201293071093072210.1155/2012/93071022675363PMC3364548 43.VenèR.LargheroP.ArenaG.SpornM.B.AlbiniA.TosettiF.Glycogen synthase kinase 3beta regulates cell death induced by synthetic triterpenoidsCancer Res.2008686987699610.1158/0008-5472.CAN-07-636218757413 44.KambleS.M.PatelH.M.GoyalS.N.NoolviM.N.MahajanU.B.OjhaS.PatilC.R.In silico Evidence for Binding of Pentacyclic Triterpenoids to Keap1-Nrf2 Protein-Protein Binding SiteComb. Chem. High Throughput Screen.20172021523410.2174/138620731966616121411182228024463 45.KouR.W.XiaB.HanR.LiZ.Q.YangJ.R.YinX.GaoY.Q.GaoJ.M.Neuroprotective effects of a new triterpenoid from edible mushroom on oxidative stress and apoptosis through the BDNF/TrkB/ERK/CREB and Nrf2 signaling pathway in vitro and in vivoFood Funct.202213121211213410.1039/D2FO02854A36321740 46.PatyarS.PrakashA.MedhiB.Dual inhibition: A novel promising pharmacological approach for different disease conditionsJ. Pharm. Pharmacol.20116345947110.1111/j.2042-7158.2010.01236.x21401597 47.WangL.LewisT.ZhangY.L.KhodierC.MageshS.ChenL.InoyamaD.ChenY.ZhenJ.HuL.Q.The identification and characterization of non-reactive inhibitor of Keap1-Nrf2 interaction through HTS using a fluorescence polarization assayProbe Reports from the NIH Molecular Libraries ProgramNational Center for Biotechnology Information (US)Bethesda, MD, USA201024260785 48.KobayashiM.YamamotoM.Molecular mechanisms activating the Nrf2-Keap1 pathway of antioxidant gene regulationAntioxid. Redox Signal.2005738539410.1089/ars.2005.7.38515706085 49.LuM.WangP.QiaoY.J.GeY.FlickingerB.MalhotracD.K.DworkinL.D.LiuZ.S.GongR.J.GSK3β-mediated Keap1-independent regulation of Nrf2 antioxidant response: A molecular rheostat of acute kidney injury to chronic kidney disease transitionRedox Biol.20192610127510129110.1016/j.redox.2019.10127531349118PMC6669347 50.QiuL.CaiC.ZhaoX.FangY.TangW.GuoC.Inhibition of aldose reductase ameliorates ethanol-induced steatosis in HepG2 cellsMol. Med. Rep.2017152732273610.3892/mmr.2017.631328447762 51.LledíasF.HansbergW.Oxidation of human catalase by singlet oxygen in myeloid leukemia cellsPhotochem. Photobiol.19997088789210.1111/j.1751-1097.1999.tb08298.x10628300 52.RenJ.ShaW.ShangS.YuanE.Hepatoprotective peptides purified from Corbicula fluminea and its effect against ethanol-induced LO2 cells injuryInt. J. Food Sci. Technol.20215635236110.1111/ijfs.14649 53.KangT.C.Nuclear Factor-Erythroid 2-Related Factor 2 (Nrf2) and Mitochondrial Dynamics/Mitophagy in Neurological DiseasesAntioxidants2020961763710.3390/antiox907061732679689PMC7402121 54.PingZ.ZhangL.F.CuiY.J.ChangY.M.JiangC.W.MengZ.Z.XuP.LiuH.Y.WangD.Y.CaoX.B.The protective effects of salidroside from exhaustive exercise-induced heart injury by enhancing the PGC-1α–NRF1/NRF2 pathway and mitochondrial respiratory function in ratsOxidative Med. Cell. Longev.2015201587682587683410.1155/2015/87682526167242PMC4488012 55.YeligarS.M.MachidaK.KalraV.K.Ethanol-induced HO-1 and NQO1 are differentially regulated by HIF-1alpha and Nrf2 to attenuate inflammatory cytokine expressionJ. Biol. Chem.2010285353593537310.1074/jbc.M110.13863620833713PMC2975160 56.El-AssalO.HongF.KimW.H.RadaevaS.GaoB.IL-6-deficient mice are susceptible to ethanol-induced hepatic steatosis: IL-6 protects against ethanol-induced oxidative stress and mitochondrial permeability transition in the liverCell. Mol. Immunol.2004120521116219169 57.DudonneS.VitracX.CoutiereP.WoillezM.MérillonJ.M.Comparative study of antioxidant properties and total phenolic content of 30 plant extracts of industrial interest using DPPH, ABTS, FRAP, SOD, and ORAC assaysJ. Agric. Food Chem.2009571768177410.1021/jf803011r19199445 58.WangM.ChenW.Y.ZhangJ.GobejishviliL.BarveS.S.McClainC.J.Joshi-BarveS.Elevated Fructose and Uric Acid through Aldose Reductase Contribute to Experimental and Human Alcoholic Liver DiseaseHepatology.2020721617163710.1002/hep.3119732086945 59.ZhouS.WangP.QiaoY.GeY.WangY.Z.QuanS.X.YaoR.ZhuangS.G.WangL.J.DuY.Genetic and pharmacologic targeting of glycogen synthase kinase 3β reinforces the Nrf2 antioxidant defense against podocytopathyJ. Am. Soc. Nephrol.2016272289230810.1681/ASN.201505056526647425PMC4978041 60.BryanH.K.OlayanjuA.GoldringC.E.ParkB.K.The Nrf2 cell defence pathway: Keap1-dependent and-independent mechanisms of regulationBiochem. Pharmacol.20138570571710.1016/j.bcp.2012.11.01623219527 61.SoniD.KumarP.GSK-3β-mediated regulation of Nrf2/HO-1 signaling as a new therapeutic approach in the treatment of movement disordersPharmacol. Rep.20227455756910.1007/s43440-022-00390-z35882765 62.YousefM.H.SalamaM.El-FawalH.A.N.AbdelnaserA.Selective GSK3β Inhibition Mediates an Nrf2-Independent Anti-inflammatory Microglial ResponseMol. Neurobiol.2022595591561110.1007/s12035-022-02923-235739410PMC9395457 63.LiG.D.BoyleJ.W.KoC.N.ZengW.WongV.K.W.WanJ.B.ChanP.W.H.MaD.L.LeungC.H.Aurone derivatives as Vps34 inhibitors that modulate autophagyActa Pharm. Sin. B2019953754410.1016/j.apsb.2019.01.01631193773PMC6543056 64.ChouM.C.ChangR.HungY.H.ChenY.C.ChiuC.H.Antrodia camphorata ameliorates high-fat-diet induced hepatic steatosis via improving lipid metabolism and antioxidative statusJ. Funct. Foods201351317132510.1016/j.jff.2013.04.019 65.TienA.J.ChienC.Y.ChenY.H.LinL.C.ChienC.T.Fruiting Bodies of Antrodia cinnamomea and Its Active Triterpenoid, Antcin K, Ameliorates N-Nitrosodiethylamine-Induced Hepatic Inflammation, Fibrosis and Carcinogenesis in RatsAm. J. Chin. Med.20174517319810.1142/S0192415X1750012428081627 66.WangL.LiW.H.ZhangR.GeY.P.YangS.D.LiuW.WuQ.P.ChengX.H.Study on Characteristics of Triterpenoids and Hepatoprotective Effects of Fruit Body of Stout Camphor Mushroom, Taiwanofungus camphoratus (Agaricomycetes), Cultivated with Apple-WoodInt. J. Med. Mushrooms202224536510.1615/IntJMedMushrooms.202204411835993961 67.HuangG.J.DengJ.S.HuangS.S.LeeC.Y.HouW.C.WangS.Y.SungP.J.KuoY.H.Hepatoprotective effects of eburicoic acid and dehydroeburicoic acid from Antrodia camphorata in a mouse model of acute hepatic injuryFood Chem.20131413020302710.1016/j.foodchem.2013.03.06123871054 68.YiZ.LiuX.LiangL.WangG.XiongZ.ZhangH.SongX.AiL.XiaY.Antrodin A from Antrodia camphorata modulates the gut microbiome and liver metabolome in mice exposed to acute alcohol intakeFood Funct.2021122925293710.1039/D0FO03345F33720247 69.HuangC.H.ChangY.Y.LiuC.W.KangW.Y.LinY.L.ChangH.C.ChenY.C.Fruiting body of Niuchangchih (Antrodia camphorata) protects livers against chronic alcohol consumption damageJ. Agric. Food Chem.2010583859386610.1021/jf100530c20192205 70.QinJ.J.ChengX.D.ZhangJ.ZhangW.D.Dual roles and therapeutic potential of Keap1-Nrf2 pathway in pancreatic cancer: A systematic reviewCell Commun. Signal.20191711510.1186/s12964-019-0435-231511020PMC6740038 71.LuM.C.ZhaoJ.LiuY.T.LiuT.TaoM.M.YouQ.D.JiangZ.Y.CPUY192018, a potent inhibitor of the Keap1-Nrf2 protein-protein interaction, alleviates renal inflammation in mice by restricting oxidative stress and NF-κB activationRedox Biol.20192610126610.1016/j.redox.2019.10126631279986PMC6614503 72.SunY.HeL.WangT.HuaW.QinH.WangJ.WangL.GuW.LiT.LiN.Activation of p62-Keap1-Nrf2 Pathway Protects 6-Hydroxydopamine-Induced Ferroptosis in Dopaminergic CellsMol. Neurobiol.2020574628464110.1007/s12035-020-02049-332770451 73.SunY.HuangJ.ChenY.ShangH.ZhangW.YuJ.HeL.XingC.ZhuangC.Direct inhibition of Keap1-Nrf2 Protein-Protein interaction as a potential therapeutic strategy for Alzheimer’s diseaseBioorganic Chem.202010310417210.1016/j.bioorg.2020.10417232890991 74.Di MartinoR.M.C.PruccoliL.BisiA.GobbiS.RampaA.MartinezA.PérezC.Martinez-GonzalezL.PaglioneM.Di SchiaviE.Novel Curcumin-Diethyl Fumarate Hybrid as a Dualistic GSK-3β Inhibitor/Nrf2 Inducer for the Treatment of Parkinson’s DiseaseACS Chem. Neurosci.2020112728274010.1021/acschemneuro.0c0036332663009PMC8009478 75.HuangY.LinX.QiaoX.JiS.LiuK.D.YehC.T.TzengY.M.GuoD.YeM.Antcamphins A-L, ergostanoids from Antrodia camphorateJ. Nat. Prod.20147711812410.1021/np400741s24387703 76.YangG.J.KoC.N.ZhongH.J.LeungC.H.MaD.L.Structure-Based Discovery of a Selective KDM5A Inhibitor that Exhibits Anti-Cancer Activity via Inducing Cell Cycle Arrest and Senescence in Breast Cancer Cell LinesCancers2019119210710.3390/cancers1101009230650517PMC6360022 77.TaylorS.WakemM.DijkmanG.AlsarrajM.NguyenM.A practical approach to RT-qPCR-Publishing data that conform to the MIQE guidelinesMethods201050S1S510.1016/j.ymeth.2010.01.00520215014 78.DaiW.ZhaoF.LiuJ.LiuH.ASCT2 Is Involved in SARS-Mediated β-Casein Synthesis of Bovine Mammary Epithelial Cells with Methionine SupplyJ. Agric. Food Chem.201968130381304510.1021/acs.jafc.9b0383331597423 79.DevlingT.W.LindsayC.D.McLellanL.I.McMahonM.HayesJ.D.Utility of siRNA against Keap1 as a strategy to stimulate a cancer chemopreventive phenotypeProc. Natl. Acad. Sci. USA20051027280728510.1073/pnas.050147510215883370PMC1091750 80.CiottiS.IulianoL.CefalùS.ComelliM.MavelliI.Di GiorgioE.BrancoliniC.GSK3β is a key regulator of the ROS-dependent necrotic death induced by the quinone DMNQCell Death Dis.202011210.1038/s41419-019-2202-031919413PMC6952365
--------------------------------------------------------------------------------
""".strip()

print(raw_text[:800])

[1] front :: Abstract
Alcoholic liver disease (ALD) is a complicated disease which can lead to hepatocellular carcinoma; however, there is a lack of satisfactory therapeutics. Dehydroeburicoic acid (DEA) (1), a triterpenoid isolated from Antrodia cinnamomea, has been reported to act against ALD, but its mechanisms of action are still not clear. In this study, we report for the first time the use of DEA (1) as a dual inhibitor of the Keap1–Nrf2 protein–protein interaction (PPI) and GSK3β in an in vitro ALD cell model. DEA (1) engages Keap1 to disrupt the Keap1–Nrf2 PPI and inhibits GSK3β to restore Nrf2 activity in a Keap1-independent fashion. DEA (1) promotes Nrf2 nuclear translocation to activate downstream antioxidant genes. Importantly, DEA (1) restores the mitochondrial dysfunction ind


In [4]:
import re, json, random
from pathlib import Path
random.seed(42)

DATA_DIR = Path("data"); DATA_DIR.mkdir(exist_ok=True)
MIN_BODY_CHARS = 200  # skip tiny sections

def clean_text(s: str) -> str:
    s = s.strip()
    s = re.sub(r"\s+\n", "\n", s)
    s = re.sub(r"[ \t]+", " ", s)
    return s

# Header patterns that are supported
HEADER_PAT_A = re.compile(r"^(.+?)\s*\|\s*idx\s*\d+/\d+\s*$", re.I)                      # "Title | idx k/n"
HEADER_PAT_B = re.compile(r"^\[\d+\]\s*(?:front|body|back)?\s*::\s*(.+?)\s*$", re.I)     # "[n] front :: Title"
HEADER_PAT_C = re.compile(r"^\d+\.\s+[A-Z].{2,}$")                                       # "1. Title"
SEP_LINE     = re.compile(r"^\s*[-_=]{5,}\s*$")                                          # ------, ______, ======

def parse_sections_from_raw(raw_text: str):
    text = (raw_text or "").replace("\r\n", "\n").replace("\r", "\n").strip()
    lines = text.splitlines()

    sections = []
    curr_title = None
    curr_buf = []

    def flush():
        nonlocal curr_title, curr_buf
        if curr_title and curr_buf:
            body = clean_text("\n".join(curr_buf).strip())
            if len(body) >= MIN_BODY_CHARS:
                sections.append({"title": clean_text(curr_title), "body": body})
        curr_title, curr_buf = None, []

    for line in lines:
        # Skip obvious separator lines
        if SEP_LINE.match(line):
            continue

        mA = HEADER_PAT_A.match(line)
        mB = HEADER_PAT_B.match(line)
        mC = HEADER_PAT_C.match(line)

        if mA or mB or mC:
            # New header → flush previous
            flush()
            if mA:
                curr_title = mA.group(1)
            elif mB:
                curr_title = mB.group(1)
            else:
                # keep the numbered title text as-is
                curr_title = line.strip()
            continue

        # Accumulate body
        if curr_title is not None:
            curr_buf.append(line)

    # last section
    flush()
    return sections

sections = parse_sections_from_raw(raw_text)
print(f"Parsed sections: {len(sections)}")
if sections:
    print("Example title:", sections[0]["title"])
    print("Body preview:", sections[0]["body"][:200], "…")
else:
    print("Still 0 sections. Paste the first ~20 lines here and we’ll tweak the patterns.")


Parsed sections: 26
Example title: Abstract
Body preview: Alcoholic liver disease (ALD) is a complicated disease which can lead to hepatocellular carcinoma; however, there is a lack of satisfactory therapeutics. Dehydroeburicoic acid (DEA) (1), a triterpenoi …


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Open instruct model just for QUESTION GENERATION (separate from the target model)
QG_MODEL = "Qwen/Qwen2.5-7B-Instruct"

bnb_qg = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

qg_tok = AutoTokenizer.from_pretrained(QG_MODEL, use_fast=True)
qg_tok.pad_token = qg_tok.eos_token
qg_tok.padding_side = "left"

qg = AutoModelForCausalLM.from_pretrained(
    QG_MODEL,
    quantization_config=bnb_qg,
    torch_dtype=torch.bfloat16,
    device_map="auto",
).eval()

print("QG model loaded:", QG_MODEL)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

QG model loaded: Qwen/Qwen2.5-7B-Instruct


In [6]:
MAX_QA_PER_SECTION = 3
MAX_NEW_TOKENS_QG  = 512
MAX_ANS_CHARS      = 350

def qg_prompt(section_text, title):
    return (
        "<s>[INST] You are a scientific question generator. "
        "Given the section below, write 2-3 QUESTION–ANSWER pairs. "
        "STRICT RULES: (1) Questions must be specific to THIS section; "
        "(2) Answers MUST be EXTRACTIVE (copy exact text spans from the section; no paraphrasing); "
        "(3) Prefer numeric findings (EC50/IC50, %, fold), mechanisms (Keap1–Nrf2, GSK3β), assays, dosages; "
        '(4) Output pure JSON list, e.g. [{"instruction":"<question>","output":"<EXACT SPAN>"}]. '
        f"Section title: {title}\n\nSECTION:\n{section_text}\n[/INST]"
    )

def generate_qas_for_section(title, body, max_items=MAX_QA_PER_SECTION, temperature=0.2):
    prompt = qg_prompt(body, title)
    inps = qg_tok(prompt, return_tensors="pt").to(qg.device)
    with torch.no_grad():
        out = qg.generate(
            **inps,
            max_new_tokens=MAX_NEW_TOKENS_QG,
            temperature=temperature,
            do_sample=(temperature>0),
        )
    text = qg_tok.decode(out[0], skip_special_tokens=True)

    # Grab the first JSON list
    m = re.search(r"\[.*\]", text, flags=re.S)
    if not m:
        return []

    try:
        items = json.loads(m.group(0))
    except Exception:
        return []

    # Enforce extractive spans + dedupe + length guard
    cleaned, seen = [], set()
    for it in items:
        q = (it.get("instruction") or "").strip()
        a = (it.get("output") or "").strip()
        if not q or not a:
            continue
        if a not in body:
            continue
        if len(a) > MAX_ANS_CHARS:
            continue
        key = (q.lower(), a.lower())
        if key in seen:
            continue
        seen.add(key)
        cleaned.append({"instruction": q, "output": a})
        if len(cleaned) >= max_items:
            break
    return cleaned


In [7]:
import os, json, time, torch
from time import perf_counter

tok = qg_tok
mdl = qg

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:128"

import re, json

def parse_qas_loose(text: str):
    s, e = text.find("["), text.rfind("]")
    chunk = text[s:e+1] if s != -1 and e != -1 and e > s else text
    chunk = chunk.replace("“", '"').replace("”", '"').replace("’", "'").replace("\n", " ")
    chunk = re.sub(r'\[\s*"question"\s*:', r'[{"question":', chunk)
    chunk = re.sub(r',\s*"question"\s*:', r'}, {"question":', chunk)
    try:
        arr = json.loads(chunk)
        if isinstance(arr, dict): 
            arr = [arr]
    except Exception:
        arr = []
        for m in re.finditer(r'"question"\s*:\s*"(.*?)"\s*,\s*"answer"\s*:\s*(\[.*?\]|\{.*?\}|".*?")', chunk):
            q = m.group(1)
            a_raw = m.group(2)
            if a_raw.startswith('"'):
                a = a_raw.strip('"')
            else:
                try:
                    a_json = json.loads(a_raw)
                    a = ", ".join(map(str, a_json)) if isinstance(a_json, list) else json.dumps(a_json, ensure_ascii=False)
                except Exception:
                    a = a_raw
            arr.append({"question": q, "answer": a})
    out = []
    for it in arr:
        if isinstance(it, dict):
            q = (it.get("question") or "").strip()
            a = (it.get("answer") or "").strip()
            if q and a:
                out.append({"question": q, "answer": a})
    return out


def truncate_text(txt: str, max_tokens: int) -> str:
    ids = tok(txt, add_special_tokens=False)["input_ids"]
    if len(ids) <= max_tokens:
        return txt
    return tok.decode(ids[:max_tokens], skip_special_tokens=True)

def gen_one_via_yours(title: str, body: str, max_inp_tokens=800):
    title_s = truncate_text(title, 128)
    body_s  = truncate_text(body,  max_inp_tokens)
    try:
        pairs = generate_qas_for_section(title_s, body_s)
    except Exception as e:
        print("ERROR inside generate_qas_for_section:", e)
        return []
    if not pairs:
        print("DEBUG empty:", {"title_tok": len(tok(title_s)["input_ids"]),
                               "body_tok":  len(tok(body_s)["input_ids"])})
    return pairs

def gen_one_direct_json(title: str, body: str,
                        max_inp_tokens=800, max_new_tokens=192):
    title_s = truncate_text(title, 128)
    body_s  = truncate_text(body,  max_inp_tokens)

    messages = [
        {"role": "system",
         "content": ("You write exam-style question–answer pairs in strict JSON. "
                     "Return a JSON array (max 3 objects). Keys must be exactly "
                     '"question" and "answer". Be concise and use only the given text.')},
        {"role": "user",
         "content": f"Title: {title_s}\n\nText:\n{body_s}\n\nReturn only JSON, starting with '[' and ending with ']'."}
    ]

    prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    with torch.inference_mode():
        enc = tok(prompt, return_tensors="pt", truncation=True,
                  max_length=max_inp_tokens+256).to(mdl.device)
        out = mdl.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=1,
            use_cache=True,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.pad_token_id
        )

    gen_ids = out[0][enc["input_ids"].shape[-1]:]
    text = tok.decode(gen_ids, skip_special_tokens=True)

    pairs = parse_qas_loose(text)
    if pairs:
      return pairs

    start = text.find("["); end = text.rfind("]")
    if start != -1 and end != -1 and end > start:
        try:
            arr = json.loads(text[start:end+1])
            cleaned = []
            for item in arr:
                if isinstance(item, dict) and "question" in item and "answer" in item:
                    q = (item["question"] or "").strip()
                    a = (item["answer"] or "").strip()
                    if q and a:
                        cleaned.append({"question": q, "answer": a})
            return cleaned
        except Exception:
            pass

    os.makedirs("qg_debug", exist_ok=True)
    fname = f"qg_debug/raw_{int(time.time()*1000)}.txt"
    with open(fname, "w", encoding="utf-8") as df:
        df.write(text)
    print("DEBUG: direct generation produced non-JSON. Saved raw to", fname)
    print("DEBUG preview:", text[:600].replace("\n", " "))
    return []


QG_JSONL = DATA_DIR / "train_qg.jsonl"
total = len(sections)
with open(QG_JSONL, "w", encoding="utf-8") as f:
    for i, sec in enumerate(sections, 1):
        t0 = perf_counter()
        pairs = gen_one_via_yours(sec["title"], sec["body"])
        if not pairs:
            pairs = gen_one_direct_json(sec["title"], sec["body"])
        for ex in pairs:
            f.write(json.dumps(ex, ensure_ascii=False) + "\n")
        torch.cuda.empty_cache()
        print(f"[{i}/{total}] {len(pairs)} QAs in {perf_counter()-t0:.1f}s")
print("Saved:", QG_JSONL)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


DEBUG empty: {'title_tok': 1, 'body_tok': 243}
[1/26] 3 QAs in 47.6s
DEBUG empty: {'title_tok': 2, 'body_tok': 384}
[2/26] 1 QAs in 48.5s
DEBUG empty: {'title_tok': 3, 'body_tok': 800}
[3/26] 2 QAs in 35.1s
DEBUG empty: {'title_tok': 27, 'body_tok': 515}
[4/26] 2 QAs in 34.4s
DEBUG empty: {'title_tok': 18, 'body_tok': 82}
DEBUG: direct generation produced non-JSON. Saved raw to qg_debug/raw_1758669530797.txt
DEBUG preview: ["question","What cell lines were used to test the cytotoxicity of DEA (1)?","answer":["LO2","HEK 293T","HepG2"]]
[5/26] 0 QAs in 10.7s
DEBUG empty: {'title_tok': 23, 'body_tok': 319}
[6/26] 3 QAs in 35.5s
DEBUG empty: {'title_tok': 41, 'body_tok': 663}
[7/26] 1 QAs in 28.5s
DEBUG empty: {'title_tok': 25, 'body_tok': 624}
[8/26] 1 QAs in 64.2s
DEBUG empty: {'title_tok': 31, 'body_tok': 311}
[9/26] 1 QAs in 16.8s
DEBUG empty: {'title_tok': 3, 'body_tok': 800}
[10/26] 1 QAs in 32.6s
DEBUG empty: {'title_tok': 8, 'body_tok': 126}
[11/26] 1 QAs in 12.9s
DEBUG empty: {'ti

In [8]:
from pathlib import Path
import json, re, unicodedata, random

INPUT = Path("data/train_qg.jsonl")            
OUTPUT_TRAIN = Path("train_clean.jsonl")  
OUTPUT_VAL = Path("val_clean.jsonl")      

VAL_FRACTION = 0.1   
MIN_Q_CHARS = 12     
MIN_A_CHARS = 6      
RANDOM_SEED = 42

def normalize_text(s: str) -> str:
    s = unicodedata.normalize("NFKC", s or "")
    s = s.replace("μ", "µ")
    s = re.sub(r"\s+", " ", s).strip()
    s = re.sub(r"\b(uM|μM)\b", "µM", s, flags=re.I)
    s = re.sub(r"\bhrs?\b", "h", s, flags=re.I)
    return s

def normalize_answer(ans):
    if isinstance(ans, str):
        return normalize_text(ans)
    if isinstance(ans, list):
        return ", ".join([normalize_answer(x) for x in ans])
    if isinstance(ans, dict):
        return "; ".join(f"{k}: {normalize_answer(v)}" for k, v in ans.items())
    return normalize_text(str(ans) if ans is not None else "")

def read_jsonl(p: Path):
    with p.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                yield json.loads(line)
            except json.JSONDecodeError:
                continue

def write_jsonl(p: Path, rows):
    with p.open("w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

def clean_records(rows):
    cleaned = []
    for r in rows:
        q = normalize_text(r.get("question", ""))
        a = normalize_answer(r.get("answer", ""))
        if len(q) < MIN_Q_CHARS or len(a) < MIN_A_CHARS:
            continue
        cleaned.append({"question": q, "answer": a})
    seen, dedup = set(), []
    for r in cleaned:
        key = r["question"].lower()
        if key not in seen:
            seen.add(key)
            dedup.append(r)
    return dedup

def train_val_split(rows, val_frac=0.1, seed=42):
    rows = list(rows)
    random.Random(seed).shuffle(rows)
    n_val = max(1, int(len(rows) * val_frac))
    return rows[n_val:], rows[:n_val]

rows = list(read_jsonl(INPUT))
rows = clean_records(rows)
train, val = train_val_split(rows, VAL_FRACTION, RANDOM_SEED)
write_jsonl(OUTPUT_TRAIN, train)
write_jsonl(OUTPUT_VAL, val)

print(f"Вкупно по чистење: {len(rows)} записи")
print(f"Train: {len(train)} → {OUTPUT_TRAIN}")
print(f"Val:   {len(val)} → {OUTPUT_VAL}")

Вкупно по чистење: 26 записи
Train: 24 → train_clean.jsonl
Val:   2 → val_clean.jsonl


In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None and tokenizer.eos_token is not None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
print("Target model ready:", MODEL_NAME)


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Target model ready: mistralai/Mistral-7B-Instruct-v0.3


In [10]:
from huggingface_hub import login
login()

In [12]:
from datasets import load_dataset

ds = load_dataset("json", data_files={
    "train": "train_clean.jsonl",
    "validation": "val_clean.jsonl"
})
print(ds)

def row_to_chat(ex):
    messages = [
        {"role": "system", "content": "Answer concisely and only based on the given question."},
        {"role": "user", "content": ex["question"]},
        {"role": "assistant", "content": ex["answer"]},
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    if tokenizer.eos_token and not text.endswith(tokenizer.eos_token):
        text += tokenizer.eos_token
    return {"text": text}

train_text = ds["train"].map(row_to_chat, remove_columns=ds["train"].column_names)
val_text   = ds["validation"].map(row_to_chat, remove_columns=ds["validation"].column_names)

print(train_text[0]["text"][:300])

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 24
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 2
    })
})


Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

<s>[INST] How did DEA (1) compare to ML334 in terms of inhibiting the Keap1–Nrf2 interaction?[/INST] DEA (1) had a 45% reduction of Keap1 co-precipitated with Nrf2 compared to 12% inhibition for ML334.</s>


In [17]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

import os
os.environ["WANDB_DISABLED"] = "true" 
os.environ["WANDB_CONSOLE"] = "off"

MAX_SEQ_LEN   = 1024
BATCH_SIZE    = 1        
GRAD_ACCUM    = 8        # effective batch = BATCH_SIZE * GRAD_ACCUM
LR             = 2e-4
EPOCHS         = 3
OUTPUT_DIR     = "qg_qwen_lora"
SEED           = 42

peft_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(base_model)
model = get_peft_model(model, peft_cfg)
model.print_trainable_parameters()

def tok_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_SEQ_LEN,
        padding=False,
        return_tensors=None,
    )

train_tok = train_text.map(tok_fn, batched=True, remove_columns=["text"])
val_tok   = val_text.map(tok_fn,   batched=True, remove_columns=["text"])

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LR,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    logging_steps=20,
    # evaluation_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    bf16=True,
    optim="paged_adamw_8bit",
    report_to=None,   # or [] on some old versions
    seed=SEED,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    # eval_dataset=val_tok,
    data_collator=collator,
)
trainer.train()

model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved LoRA adapter to", OUTPUT_DIR)


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 41,943,040 || all params: 7,289,966,592 || trainable%: 0.5754


Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


Saved LoRA adapter to qg_qwen_lora


In [19]:
from peft import PeftModel
from transformers import TextStreamer
import torch, gc

# reuse the base_model + tokenizer you already have in memory
# (created before training)
gc.collect(); torch.cuda.empty_cache()

ft = PeftModel.from_pretrained(base_model, OUTPUT_DIR).eval()
streamer = TextStreamer(tokenizer, skip_special_tokens=True)

def chat(user_msg, max_new_tokens=256, temperature=0.2):
    messages = [
        {"role": "system", "content": "Answer concisely and only based on the question."},
        {"role": "user", "content": user_msg},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(ft.device)
    with torch.no_grad():
        _ = ft.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=(temperature > 0),
            temperature=temperature,
            streamer=streamer,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

# quick test
chat("Explain the dual mechanism of DEA (1) in ALD according to the paper.")

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Answer concisely and only based on the question.

Explain the dual mechanism of DEA (1) in ALD according to the paper. DEA (1) inhibits ALD by suppressing the activity of ALDH2 and ALDH1A1, and it also activates Keap1–Nrf2 interaction to induce Nrf2 nuclear translocation.

Citation: Li, Y., et al. (2019). DEA (1) protects against alcoholic liver disease by inhibiting ALDH2 and ALDH1A1 and activating Keap1–Nrf2 interaction. Journal of Medicinal Chemistry, 62(20), 10760–10771. DOI: 10.1021/acs.jmedchem.9b01289.
